<a href="https://colab.research.google.com/github/comradyo/kis/blob/main/lr2/lr-2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Лабораторная работа 2. Методы анализа параметров компонентов моделей функционирования КИС

## Задание 1.

Заданы три независимые целочисленные неотрицательные случайные величины $X_1, X_2, X_3$, необходимо найти:
- математическое ожидание, 
- дисперсию, 
- среднее квадратическое отклонение, 
- коэффициент вариации 

случайной величины $X_1+X_2+X_3$ численно и теоретически (из определения и с использованием производящей функции).
В таблице ниже представлены соответствия значений случайной величины и их вероятности. 

In [ ]:
Variant<-8
set.seed(Variant) 
X1<-sample(c(1:20),5)
X2<-sample(c(3:100),5)
X3<-sample(c(0:40),5)
pp1<-runif(5)
p1<-pp1/sum(pp1)
pp2<-runif(5)
p2<-pp2/sum(pp2)
pp3<-runif(5)
p3<-pp3/sum(pp3)

In [ ]:
View(data.frame(X1, X2, X3, p1, p2, p3))

X1,X2,X3,p1,p2,p3
<int>,<int>,<int>,<dbl>,<dbl>,<dbl>
20,44,34,0.1417121,0.04319863,0.25161206
2,3,5,0.2136064,0.27043601,0.37736297
15,95,21,0.0746814,0.23317449,0.07386225
12,52,10,0.2304178,0.36122455,0.15866802
7,5,3,0.3395822,0.09196632,0.13849469


In R one can have objects of type "expression". An expression contains one or more statements. A **statement** is a syntactically correct collection of tokens. **Expression** objects are special
language objects which contain parsed but unevaluated R statements. The main difference is
that an expression object can contain several such expressions. Objects of type "expression" are only evaluated when explicitly (явно) passed to eval, whereas
other language objects may get evaluated in some unexpected cases.

Производящая функция случайной величины:
$$P_{\zeta}(z)=\sum_{n=0}^\infty p_nz^n$$
Свойства производящей функции

1. $$P_{\zeta}(1)=1$$
2. $$M(\zeta)=P'_{\zeta}(z)|_{z=1}$$
3. $$D(\zeta)=P''_{\zeta}(z)|_{z=1}+P'_{\zeta}(z)|_{z=1}-\left(P'_{\zeta}(z)\right)^2|_{z=1}$$
4. $$p_n=\frac{1}{n!}\frac{d^n P_{\zeta}(z)}{dz^n}|_{z=0}$$
5. Если $\zeta_1, \zeta_2$ две независимые целочисленные случайные неотрицательные величины, то
$$P_{\zeta_1+\zeta_2}(z)=P_{\zeta_1}(z)\cdot P_{\zeta_2}(z)$$
6. Если $\zeta=\sum_{i=1}^{v}\zeta_i$, где $v$ - целочисленная неотрицательная случайная величина, $\forall i: \zeta_i$ - одинаково распределенные независимые целочисленные неотрицательные случайные величины, то
$$P_{\zeta}(z)=P_v\left(P_{\zeta_i}(z)\right)$$
Если $\zeta_i$ непрерывные случайные величины, то
$$\beta_{\zeta}(s)=P_v\left(\beta_{\zeta_i}(s)\right)$$

Воспользуемся этими свойствами для выполнения задания.

Проведем теоретическое моделирование в среде R:

In [ ]:
# производная n-го порядка
DD <- function(expr, name, order = 1) {
  if(order < 1) stop("'order' must be >= 1")
  if(order == 1) return(D(expr, name))
  else return(DD(D(expr, name), name, order - 1))
}

In [ ]:
Px1<-""; Px2<-""; Px3<-""

for (i in (1:5)) {
  Px1<-paste(Px1, "+",p1[i],"*z^",X1[i])
  Px2<-paste(Px2, "+",p2[i],"*z^",X2[i])
  Px3<-paste(Px3, "+",p3[i],"*z^",X3[i])
}

Px_temp <-paste("(",Px1,")*(",Px2,")*(",Px3,")")

In [ ]:
# string -> expression 
Px_ <- parse(text=Px_temp)
# Evaluate (вычислить) an R expression in a specified environment.
# В данном случае Px_ - это expression, в котором 'z' - аргумент.
Px <- function(z) {eval(Px_)} # Производящая функция

DPx_ <- D(Px_, 'z')
DDPx_ <- DD(Px_, 'z', 2)

In [ ]:
# Мат. ожидание
Mx <- function(z) {eval(DPx_)}
# дисперсия
Dx <- function(z) {eval(DDPx_) + eval(DPx_) - eval(DPx_) ^ 2}
# среднеквадратичное отклонение
MSD <- function(z) {sqrt(Dx(z))}
# коэффициент вариации
COV <- function(z) {MSD(z) * 100 / Mx(z)}

In [ ]:
Px(1)
Mx(1)
Dx(1)
MSD(1)
COV(1)

[1] 1

[1] 67.62579

[1] 1411.665

[1] 37.57213

[1] 55.55887

Проведем экспериментальное моделирование в среде R:

In [ ]:
#Экспериментальная часть

N <- 100000
xx1 <- c()
xx2 <- c()
xx3 <- c()
for (i in c(1:N)) {
  f<-runif(1)
  p_left <- 0
  p_right <- p1[1]
  for (j in c(1:length(p1))) {
    if ((f > p_left) & (f < p_right))
      xx1 <- c(xx1, X1[j])
    if (j != length(p1)) {
      p_left <- p_right
      p_right <- p_right + p1[j + 1]
    }
  }
  
  f<-runif(1)
  p_left <- 0
  p_right <- p2[1]
  for (j in c(1:length(p2))) {
    if ((f > p_left) & (f < p_right))
      xx2 <- c(xx2, X2[j])
    if (j != length(p2)) {
      p_left <- p_right
      p_right <- p_right + p2[j + 1]
    }
  }
  
  f<-runif(1)
  p_left <- 0
  p_right <- p3[1]
  for (j in c(1:length(p3))) {
    if ((f > p_left) & (f < p_right))
      xx3 <- c(xx3, X3[j])
    if (j != length(p3)) {
      p_left <- p_right
      p_right <- p_right + p3[j + 1]
    }
  }
}

print(mean(xx1 + xx2 + xx3))
print((sd(xx1 + xx2 + xx3)) ^ 2)

[1] 67.54393
[1] 1406.668


## Задание 2.

Заданы три независимые целочисленные неотрицательные случайные величины $X_1, X_2, X_3$, распределенные по закону Пуассона, необходимо найти:

1. математическое ожидание, 
2. дисперсию, 
3. среднее квадратическое отклонение
4. коэффициент вариации 

Случайной величины $X_1+X_2+X_3$ численно и теоретически.

Параметры закона Пуассона для $X_1, X_2, X_3$ определить в соответствии с вариантом.

In [ ]:
Variant<-8
set.seed(Variant) 
runif(3)

[1] 0.4662952 0.2078233 0.7996580

$\lambda_1=0.466295241378248$ 

$\lambda_2=0.207823317032307$

$\lambda_3=0.799657957628369$


Производящая функция распределения случайной величины, распределенной по закону Пуассона:

$$P_{\zeta}(z)=\sum_{n=0}^\infty p_nz^n$$, 
где $p_n=\frac{\lambda^n}{n!}e^{-\lambda}$ — вероятность того, что случайная величина $\zeta$ примет значение $n$.

Тогда $$P_{\zeta}(z)=\sum_{n=0}^\infty \frac{\lambda^n}{n!} e^{-\lambda}z^n = e^{-\lambda} \sum_{n=0}^\infty \frac{(\lambda z)^n}{n!} = e^{-\lambda} e^{\lambda z} = e^{\lambda (z-1)} $$

Из свойств производящей функции распределения случайной величины:

$$P_{X_1+X_2+X_3}(z)=P_{X_1}(z) \cdot P_{X_2}(z) \cdot P_{X_3}(z) = e^{\lambda_1 (z-1) + \lambda_2 (z-1) + \lambda_3 (z-1)} = e^{(z-1)(\lambda_1 + \lambda_2 + \lambda_3)} = e^{\lambda(z-1)}$$

, где $\lambda = \lambda_1 + \lambda_2 + \lambda_3$

Тогда:

$$ P'_{\zeta}(z) = (e^{\lambda(z-1)})' = \lambda e^{\lambda(z-1)} $$
$$ P''_{\zeta}(z) = (P'_{\zeta}(z))' = (\lambda e^{\lambda(z-1)})' = \lambda^2 e^{\lambda(z-1)} $$
$$M(\zeta)=P'_{\zeta}(z)|_{z=1} = \lambda$$
$$D(\zeta)=P''_{\zeta}(z)|_{z=1}+P'_{\zeta}(z)|_{z=1}-\left(P'_{\zeta}(z)\right)^2|_{z=1} = \lambda^2 + \lambda - \lambda^2 = \lambda $$

$$\sigma = \sqrt{D} = \sqrt{\lambda}$$

$$c_v=\frac{\sigma}{M(\zeta)} \cdot 100 = \frac{\sqrt{\lambda}}{\lambda}\cdot 100 = \frac{100}{\sqrt{\lambda}}$$

Проведем теоретическое моделирование в среде R:

In [5]:
# Теоретическая часть

l1 <- 0.466295241378248
l2 <- 0.207823317032307
l3 <- 0.799657957628369
l <- l1 + l2 + l3
M <- l
D <- l
sigm <- sqrt(l)
c_v <- 100 / sqrt(l)
M
D
sigm
c_v

[1] 1.473777

[1] 1.473777

[1] 1.213992

[1] 82.37287

Проведем экспериментальное моделирование в среде R:

In [6]:
# Экспериментальная часть

N<-10000000
l1 <- 0.466295241378248
l2 <- 0.207823317032307
l3 <- 0.799657957628369
X1<-rpois(N, l1)
X2<-rpois(N, l2)
X3<-rpois(N, l3)
X<-X1+X2+X3
print(mean(X))
print(var(X))
print(sd(X))
print(sd(X)/mean(X))

[1] 1.474135
[1] 1.474329
[1] 1.21422
[1] 0.8236826


## Задание 3.
Обработка сообщения в специализированной ВС осуществляется $K$ последовательными программами. Длительность работы
каждой программы представляет собой случайную величину, распределенную экспоненциально со средним значением $T=\frac{1}{\lambda}$. 

Найти: 
1. Преобразование Лапласа-Стилтьеса распределения длительности обработки сообщения в ВС
2. Математическое ожидание распределения 
3. Дисперсию распределения.

Построить модель и провести численный расчет.

In [18]:
Variant<-8
lambda<-runif(7)[7]
lambda

[1] 0.4605695

Функция распределения вероятностей и плотность распределения случайной величины $ \zeta $, распределенной по экспоненциальному закону распределения: 

$$ P(\zeta < t) = F(t) = 1-e^{-\lambda t} $$
$$ f(t) = \lambda e ^{-\lambda t} $$

Найдем преобразование Лапласа-Стилтьеса распределения длительности обработки сообщения одной из программ:

$$ B(F(t)) = \beta(s) = \int_0^\infty e^{-st} dF(t) = \int_0^\infty e^{-st} d(1-e^{-\lambda t}) = $$ 
$$ = \lambda \int_0^\infty e^{-st} e^{-\lambda t} dt = \lambda \int_0^\infty e^{t(-s-\lambda)} dt = \frac{\lambda}{-s-\lambda} \cdot e^{t(-s-\lambda)} |_0^\infty = $$
$$ = \frac{\lambda}{-s-\lambda} \cdot \frac{1}{e^{t(s+\lambda)}|_0^\infty} = 0 - \frac{\lambda}{-s-\lambda} = \frac{\lambda}{s+\lambda} $$

Найдем преобразование Лапласа-Стилтьеса распределения длительности обработки сообщения одной из программ:

Преобразование Лапласа-Стильеса для $ T = \sum_{i=1}^{k} T_i $:
$$ \beta_T(s)=\prod_{i=1}^k \beta_{T_i}(s) = (\frac{\lambda}{s+\lambda}) ^ k = \beta(s) $$

Вычислим $\beta'(s) $ и $ \beta''(s) $:

$$ \beta'(s) = (\frac{\lambda^k}{(s+\lambda)^k})' = - \frac{\lambda^k \cdot k}{(s+\lambda)^{k+1}} $$

$$ \beta''(s) = (- \frac{\lambda^k \cdot k}{(s+\lambda)^{k+1}})' = \frac{\lambda^k \cdot k \cdot (k+1) }{(s+\lambda)^{k+2}} $$

Найдем математическое ожидание распределения:
$$M(\zeta)=-\beta'(0) = \frac{k}{\lambda}$$

Найдем дисперсию распределения:
$$D(\zeta)=\beta''(0)-\left(\beta'(0)\right)^2 = \frac{k(k+1)}{\lambda^2} - \frac{k^2}{\lambda^2} = \frac{k}{\lambda^2} $$




Проведем теоретическое моделирование в среде R:

In [23]:
# Теоретическая часть

k <- 10
l <- lambda

m <- k/l
d <- k/(l*l)

m
d

[1] 21.71225

[1] 47.14219

Проведем экспериментальное моделирование в среде R:



In [24]:
# Экспериментальная часть

N <- 100000
tmp <- c()
for (i in c(1:k)) {
  if (i == 1) {
    tmp <- rexp(N, lambda)
  } else {
    tmp <- tmp + rexp(N, lambda)
  }
}

mx <- mean(tmp)
mx
s <- sd(tmp)
dx <- s^2
dx
k <- (res_exp_s/res_exp_mx) * 100

[1] 21.69226

[1] 47.34003

## Задание 4.

Доказать, что сумма величин, распределенных по закону Пуассона с параметрами $\alpha_1, \alpha_2,\ldots$ , распределена по закону Пуассона, также обосновать это экспериментально.

Доказательство аналогично доказательству в пункте 2, но вместо суммы 3-х величин мы берем сумму k величин:

$$P_{\sum_{i=1}^k X_i}(z)=\prod_{i=1}^k P_{X_i}(z) = e^{\sum_{i=1}^k\lambda_i (z-1)} = e^{(z-1) \cdot \sum_{i=1}^k\lambda_i} = e^{\lambda(z-1)}$$

, где $\lambda = \sum_{i=1}^k\lambda_i$

Дальнейшие рассуждения аналогичны рассуждениям в задании 2, в итоге мы получаем:
$$M(\zeta) = \lambda$$
$$D(\zeta) = \lambda $$

Проведем теоретическое моделирование в среде R:

In [34]:
# Теоретическая часть

Variant<-8
k <- 10
lambdas<-runif(k)
lambdas

l <- sum(lambdas)
m <- l
d <- l
m
d

[1] 0.9485486 0.8745019 0.1629481 0.3942089 0.3030835 0.3146561 0.6809139
 [8] 0.1143634 0.5370875 0.2528945

[1] 4.583206

[1] 4.583206

Проведем экспериментальное моделирование в среде R:

In [35]:
# Экспериментальная часть

k <- 10
N <- 1000000
tmp <- c()
for (i in c(1:k)) {
  if (i == 1) {
    tmp <- rpois(n=N, lambda=lambdas[i])
  } else {
    tmp <- tmp + rpois(n=N, lambda=lambdas[i])
  }
}

mx <- mean(tmp)
mx
s <- sd(tmp)
dx <- s^2
dx

[1] 4.581551

[1] 4.585612

## Задание 5.

Серверная станция состоит из двух модулей питания. Время безотказной работы каждого из них распределено по показательному закону с одинаковыми параметрами:
$$f(t)=\lambda e^{-\lambda t}$$
$t$ - время работы одного модуля до первого отказа.
Второй модуль питания включается сразу же после отказа первого. 

Определить: 
1. Плотность вероятности времени безотказной работы всей системы.
2. Математическое ожидание времени безотказной работы.
3. Дисперсию времени безотказной работы.

In [36]:
Variant<-8
lambda<-runif(100)[77]
lambda

[1] 0.8706341

Обозначим: $t_1$ — время безотказной работы первого модуля, $t_2$ — время безотказной работы второго модуля, $t = t_1 + t_2$ — время безотказной работы системы в целом.

Если две независимые случайные величины $\zeta_1, \zeta_2$ имеют функции преобразования Лапласа-Стилтьеса $\beta_1(s),\beta_2(s)$, то случайная величина $\zeta_1+\zeta_2$ имеет преобразование Лапласа-Стилтьеса $\beta_1(s)\cdot\beta_2(s)$

Если заданы две независимые неотрицательные случайные величины $\zeta_1, \zeta_2$, то функция плотности распределения вероятностей композиции в виде суммы этих случайных величин $\zeta=\zeta_1+\zeta_2$ выразится:
$$f_{\zeta}=\int_{A(\zeta)}^{B(\zeta)} f_{\zeta_1}(\zeta_1)\cdot f_{\zeta_2}(\zeta-\zeta_1)d\zeta_1$$

Пределы интегрирования вычисляются следующим образом:
$$A(\zeta)=a+\frac{\zeta-(a+d)+|\zeta-(a+d)|}{2}$$
$$B(\zeta)=b+\frac{\zeta-(b+c)-|\zeta-(b+c)|}{2}$$
$$a\leq \zeta_1\leq b, c\leq \zeta_2\leq d$$

В рамках нашей задачи, $t_1, t_2, t = t_1 + t_2$ являются случайными величинами, распределенными по показательному закону. Применим указанное ранее свойство для определения функции плотности $t$:

$$f_t = \int_{A(t)}^{B(t)} f_{t_1}(t_1) \cdot f_{t_2}(t - t_1) dt_1 = \int_{A(t)}^{B(t)} \lambda_1 e^{-\lambda_1 t_1} \cdot \lambda_2 e^{-\lambda_2 (t-t_1)} dt_1 = \lambda_1 \lambda_2 \int_{A(t)}^{B(t)}  e^{-\lambda_1 t_1 -\lambda_2 t +\lambda_2 t_1} \cdot dt_1 = \lambda^2 \int_{A(t)}^{B(t)}  e^{-\lambda t} \cdot dt_1 = \lambda^2 e^{-\lambda t} \cdot t_1 |_{A(t)}^{B(t)} $$

Определим пределы интегрирования:

$$a = 0, b = \infty, c = 0, d = \infty $$
$$A(t)=a+\frac{t-(a+d)+|t-(a+d)|}{2}=\frac{t-d+|t-d|}{2}$$
$$B(t)=b+\frac{t-(b+c)-|t-(b+c)|}{2}=b+\frac{t-b-|t-b|}{2}$$
$$ 0 \leq t_1 \leq \infty, 0 \leq t_2 \leq \infty$$

Раскроем модули, учтем, что $0 \leq t < \infty$:

$$A(t)=\frac{t-d-t+d}{2}=0$$
$$B(t)=b+\frac{t-b+t-b}{2}=b+t-b=t$$
$$ f_t = \lambda^2 e^{-\lambda t} \cdot t_1 |_{A(t)}^{B(t)} = \lambda^2 e^{-\lambda t} \cdot t $$

Определим математическое ожидание и дисперсию времени безотказной работы:
$$ \beta(s) = \int_0^\infty e^{-st} dF(t) = \int_0^\infty e^{-st} \lambda^2 e^{-\lambda t} t \cdot dt = \lambda^2 \int_0^\infty e^{-t(s+\lambda)} t \cdot dt = \frac{\lambda^2}{(s+\lambda)^2}  \int_0^\infty -t(s+\lambda) e^{-t(s+\lambda)} \cdot d(-t(s+\lambda)) = $$

$$ = \frac{\lambda^2}{(s+\lambda)^2}  \int_0^{-\infty} w e^w dw = \frac{\lambda^2}{(s+\lambda)^2} (x-1)e^x |_0^{-\infty} = \frac{\lambda^2}{(s+\lambda)^2} $$

$$ \beta'(s) = (\frac{\lambda^2}{(s+\lambda)^2})' = -\frac{2\lambda^2}{(s+\lambda)^3} $$

$$ \beta''(s) = (-\frac{2\lambda^2}{(s+\lambda)^3})' = \frac{6\lambda^2}{(s+\lambda)^4} $$

$$M(t)=-\beta'(0) = \frac{2}{\lambda}$$
$$D(t)=\beta''(0)-\left(\beta'(0)\right)^2 = \frac{6}{\lambda^2} - \frac{4}{\lambda^2} = \frac{2}{\lambda^2}$$



Проведем теоретическое моделирование в среде R:

In [42]:
# Теоретическая часть

m <- 2/lambda
d <- 2/(lambda * lambda)
m
d


[1] 2.297176

[1] 2.63851

Проведем экспериментальное моделирование в среде R:

In [43]:
# Экспериментальная часть

sum <- c(0)
N <- 10000
for (i in c(1:N)) {
  sum <- c(sum, sum(rexp(2, lambda)))
}

mx <- mean(sum)
mx
s <- sd(sum)
dx <- s^2
dx

[1] 2.300354

[1] 2.637542